In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import transformers
from transformers import LayoutLMForTokenClassification,\
    LayoutLMTokenizer, AdamW, LayoutLMv2Processor, LayoutLMv2ForTokenClassification
# from tensordict import TensorDict
from datasets import load_dataset
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import ImageDraw, Image
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import logging
# from torchvision.transforms import PILToTensor
from torcheval.metrics.functional import multiclass_f1_score, \
    multiclass_accuracy

import os 
if 'notebooks' in os.getcwd():
    os.chdir("..")
    
from src.preprocessing.make_dataset import ImageLayoutDataset

/home/pedro/Desktop/Polytechnique/few-shot-learning-transformers/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

import io
import json

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload


In [4]:
def upload_basic():
    """Insert new file.
    Returns : Id's of the file uploaded

    Load pre-authorized user credentials from the environment.
    TODO(developer) - See https://developers.google.com/identity
    for guides on implementing OAuth2 for the application.
    """
    creds, _ = google.auth.default()

    try:
        # create drive api client
        service = build('drive', 'v3', credentials=creds)

        file_metadata = {'name': 'download.jpeg'}
        media = MediaFileUpload('download.jpeg',
                                mimetype='image/jpeg')
        # pylint: disable=maybe-no-member
        file = service.files().create(body=file_metadata, media_body=media,
                                      fields='id').execute()
        print(F'File ID: {file.get("id")}')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

    return file.get('id')



In [37]:
! gcloud auth login --enable-gdrive-access --update-adc

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=V7yU8EorhYdiwLZrrL1SAu6o4KJzQM&access_type=offline&code_challenge=xkV9JlHyerrTqnC7QMcA7ASPfx7iJqDj0JYx3_n971o&code_challenge_method=S256


Application default credentials (ADC) were updated.

You are now logged in as [pedrolmssilva@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [38]:
creds, _ = google.auth.default(
    scopes= ["https://www.googleapis.com/auth/drive.file"]
)

In [39]:
creds.scopes

In [98]:
try:
    # create drive api client
    service = build('drive', 'v3', credentials=creds)

    file_metadata = {
        'name': 'model.pt',
        "parents": ["1TW9wrG-7_1YTjbc8JaAJ2bYDESN3owdO"]
    }
    media = MediaFileUpload('data/model.pt',
                            mimetype='text/plain')
    # pylint: disable=maybe-no-member
    file = service\
        .files().\
        create(
            body=file_metadata, 
            media_body=media,
            fields='id'
        ).execute()
    print(F'File ID: {file.get("id")}')

except HttpError as error:
    print(F'An error occurred: {error}')
    file = None

File ID: 1y5hkcTQE5NlioxaPk2Lo9JqExBBMkjQD


In [99]:
try:
    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API
    results = service\
        .files()\
        .list(
            pageSize=10, 
            fields="nextPageToken, files(id, name)"
        ).execute()
    
    items = results.get('files', [])

    if not items:
        print('No files found.')

    else:

        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f'An error occurred: {error}')


Files:
model.pt (1y5hkcTQE5NlioxaPk2Lo9JqExBBMkjQD)
Untitled0.ipynb (1jCXO4ci1ycC0ZELjm7xnbK1WPb-164EL)
X forum eleves contact (1MVZSYrWOh0-2_-ZGiSgLXnE3PrGVjKHbogk8mICRFXM)
Jeudi 05/09 (1CWibnlKRNzWxjxuDKehLSgekQFgLJyDRT0k35C2vjv4)
models (175HZ7NCBjMBQ6pyo8zpMNYizrGFIUbQ9)
train.log (1jCr9Z1OJnbMG8_29SXsJJE82gbo70Ns2)
train.log (14dIeRxDBk5_8wNZnhOOvWj2HTXocCAso)
Tracabililité vacances de Toussaint (10WCgjGZ8HVG0fXOxNn-MNCfQj1cBFexy8pyK1GZbqVc)
elements of statistical learning (1C1ywy0ud7QBS6T9ZKELcI2tKJqavDR-F)
Weatherwax_Epstein_Hastie_Solution_Manual.pdf (1xLcY1lIRhYc70BTcJOwUPFZr1FwZ8kLe)


In [3]:
try:
    # create drive api client
    service = build('drive', 'v3', credentials=creds)

    file_id = "1y5hkcTQE5NlioxaPk2Lo9JqExBBMkjQD"

    # pylint: disable=maybe-no-member
    request = service.files().get_media(fileId=file_id)
    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(F'Download {int(status.progress() * 100)}.')

except HttpError as error:
    print(F'An error occurred: {error}')
    file = None
    

NameError: name 'creds' is not defined

In [118]:
m = file.getvalue()

In [72]:
model = LayoutLMForTokenClassification.from_pretrained(
    'microsoft/layoutlm-base-uncased',
    num_labels=5
)

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [97]:
torch.save(model.state_dict(), "data/model.pt")

In [71]:
file.readlines

[]

In [91]:
from pydrive.drive import GoogleDrive

# Create GoogleDrive instance with authenticated GoogleAuth instance.
drive = GoogleDrive(creds)

In [ ]:
drive.

In [96]:
drive.auth

In [ ]:
from cola

In [85]:
(model.state_dict())

odict_keys(['layoutlm.embeddings.word_embeddings.weight', 'layoutlm.embeddings.position_embeddings.weight', 'layoutlm.embeddings.x_position_embeddings.weight', 'layoutlm.embeddings.y_position_embeddings.weight', 'layoutlm.embeddings.h_position_embeddings.weight', 'layoutlm.embeddings.w_position_embeddings.weight', 'layoutlm.embeddings.token_type_embeddings.weight', 'layoutlm.embeddings.LayerNorm.weight', 'layoutlm.embeddings.LayerNorm.bias', 'layoutlm.encoder.layer.0.attention.self.query.weight', 'layoutlm.encoder.layer.0.attention.self.query.bias', 'layoutlm.encoder.layer.0.attention.self.key.weight', 'layoutlm.encoder.layer.0.attention.self.key.bias', 'layoutlm.encoder.layer.0.attention.self.value.weight', 'layoutlm.encoder.layer.0.attention.self.value.bias', 'layoutlm.encoder.layer.0.attention.output.dense.weight', 'layoutlm.encoder.layer.0.attention.output.dense.bias', 'layoutlm.encoder.layer.0.attention.output.LayerNorm.weight', 'layoutlm.encoder.layer.0.attention.output.LayerNorm